In [26]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools
using Base.Threads
using LinearAlgebra

## Funzione da ottimizzare

In [27]:
function planar_arrangement_1( V,copEV,sigma::Lar.Chain=spzeros(Int8,0),
                               return_edge_map::Bool=false,multiproc::Bool=false)
	# data structures initialization
	edgenum = size(copEV, 1)
	edge_map = Array{Array{Int, 1}, 1}(undef,edgenum)
	rV = Lar.Points(zeros(0, 2))
	rEV = SparseArrays.spzeros(Int8, 0, 0)
	finalcells_num = 0

	# spaceindex computation
	model = (convert(Lar.Points,V'),Lar.cop2lar(copEV))
	bigPI = spaceindex(model::Lar.LAR)

    
        # sequential (iterative) processing of edge fragmentation
        for i in 1:edgenum
            v, ev = frag_edge(V, copEV, i, bigPI)
            newedges_nums = map(x->x+finalcells_num, collect(1:size(ev, 1)))
            edge_map[i] = newedges_nums
            finalcells_num += size(ev, 1)
            rV = convert(Lar.Points, rV)
            rV, rEV = skel_merge(rV, rEV, v, ev)
        end
    
    # merging of close vertices and edges (2D congruence)
    V, copEV = rV, rEV
    V, copEV = merge_vertices!(V, copEV, edge_map)
	return V,copEV,sigma,edge_map
end


planar_arrangement_1 (generic function with 4 methods)

## Dipendenze della funzione

In [28]:
function frag_edge(V, EV::Lar.ChainOp, edge_idx::Int, bigPI)
    alphas = Dict{Float64, Int}()
    edge = EV[edge_idx, :]
    verts = V[edge.nzind, :]
    for i in bigPI[edge_idx]
        if i != edge_idx
            intersection = intersect_edges(V, edge, EV[i, :])
            for (point, alpha) in intersection
                verts = [verts; point]
                alphas[alpha] = size(verts, 1)
            end
        end
    end
    alphas[0.0], alphas[1.0] = [1, 2]
    alphas_keys = sort(collect(keys(alphas)))
    edge_num = length(alphas_keys)-1
    verts_num = size(verts, 1)
    ev = SparseArrays.spzeros(Int8, edge_num, verts_num)
    for i in 1:edge_num
        ev[i, alphas[alphas_keys[i]]] = 1
        ev[i, alphas[alphas_keys[i+1]]] = 1
    end
    return verts, ev
end


"""
    intersect_edges(V::Lar.Points, edge1::Lar.Cell, edge2::Lar.Cell)
Intersect two 2D edges (`edge1` and `edge2`).
"""
function intersect_edges(V::Lar.Points, edge1::Lar.Cell, edge2::Lar.Cell)
    err = 10e-8

    x1, y1, x2, y2 = vcat(map(c->V[c, :], edge1.nzind)...)
    x3, y3, x4, y4 = vcat(map(c->V[c, :], edge2.nzind)...)
    ret = Array{Tuple{Lar.Points, Float64}, 1}()

    v1 = [x2-x1, y2-y1];
    v2 = [x4-x3, y4-y3];
    v3 = [x3-x1, y3-y1];
    ang1 = dot(normalize(v1), normalize(v2))
    ang2 = dot(normalize(v1), normalize(v3))
    parallel = 1-err < abs(ang1) < 1+err
    colinear = parallel && (1-err < abs(ang2) < 1+err || -err < norm(v3) < err)
    if colinear
        o = [x1 y1]
        v = [x2 y2] - o
        alpha = 1/dot(v,v')
        ps = [x3 y3; x4 y4]
        for i in 1:2
            a = alpha*dot(v',(reshape(ps[i, :], 1, 2)-o))
            if 0 < a < 1
                push!(ret, (ps[i:i, :], a))
            end
        end
    elseif !parallel
        denom = (v2[2])*(v1[1]) - (v2[1])*(v1[2])
        a = ((v2[1])*(-v3[2]) - (v2[2])*(-v3[1])) / denom
        b = ((v1[1])*(-v3[2]) - (v1[2])*(-v3[1])) / denom

        if -err < a < 1+err && -err <= b <= 1+err
            p = [(x1 + a*(x2-x1))  (y1 + a*(y2-y1))]
            push!(ret, (p, a))
        end
    end
    return ret
end


"""
    merge_vertices!(V::Lar.Points, EV::Lar.ChainOp, edge_map, err=1e-4)
Merge congruent vertices and edges in `V` and `EV`.
"""
function merge_vertices!(V::Lar.Points, EV::Lar.ChainOp, edge_map, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    newverts = zeros(Int, vertsnum)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    kdtree = KDTree(permutedims(V))

    # merge congruent vertices
    todelete = []
    i = 1
    for vi in 1:vertsnum
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]

    # merge congruent edges
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
    # maps pairs of vertex indices to edge index
    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    # builds `edge_map`
    for ei in 1:nedgenum
        nEV[ei, collect(nedges[ei])] .= 1
        etuple2idx[nedges[ei]] = ei
    end
    for i in 1:length(edge_map)
        row = edge_map[i]
        row = map(x->edges[x], row)
        row = filter(t->t[1]!=t[2], row)
        row = map(x->etuple2idx[x], row)
        edge_map[i] = row
    end
    # return new vertices and new edges
    return Lar.Points(nV), nEV
end

function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	# se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
	dim = size(V,1)
	#PARALLELIZZO LA CREAZIONE DEI CELLPOINTS
	n=length(CV)
	cellpoints = Array{Array{Float64,2}}(undef,n)
	@inbounds @threads for k=1:n
		cellpoints[k] = V[:,CV[k]]::Lar.Points
	end
	#PARALLELIZZO LA CREAZIONE DEI BOUNDING BOXES
	bboxes = Array{Array{Float64,2}}(undef,n)
	@inbounds @threads for k=1:n
		bboxes[k] = hcat(boundingbox(cellpoints[k])...)
	end
	coverXYZ= Array{Array{Array{Int64,1},1}}(undef,dim)
	#Per ogni asse x=1, y=2, z=3.....
	@threads for i=1:dim
		boxdict = coordintervals(i,bboxes)
		#Creo interval tree sull'asse i
		intTree = IntervalTrees.IntervalMap{Float64, Array}()
		@inbounds for (key,boxset) in boxdict
			intTree[tuple(key...)] = boxset
		end
		coverXYZ[i] = boxcovering(bboxes, i, intTree)     
	end
	spaceindex = Array{Array{Any,1}}(undef,length(bboxes))
	@inbounds @threads for i=1:n
		spaceindex[i] = intersect((coverXYZ[1][i],coverXYZ[2][i])...)
	end
	if(dim==3)
		@inbounds @threads for i=1:n
			spaceindex[i] = intersect((spaceindex[i],coverXYZ[3][i])...)
		end
	end
	@inbounds @simd for k=1:length(spaceindex)
		spaceindex[k] = setdiff(spaceindex[k],[k])
	end
	return spaceindex
end


#Questa funzione prende in input insieme di vertici e calcola il loro bounding box.
#Fa cioè il minimo e il massimo delle loro coordinate su tutti gli assi.
function boundingbox(vertices::Lar.Points)
	d=size(vertices)[1]
	numPoints=size(vertices)[2]
	#inizializzo gli array da ritornare [xMin, yMin, zMin] e [xMax, yMax, zMax]
	mins = zeros(d,1)
	maxs = zeros(d,1)
	for i=1:d
		mins[i]=vertices[i]
		maxs[i]=vertices[i]
	end
	@threads for i=2:numPoints
		@threads for j=1:d
			if(vertices[j+d*(i-1)] > maxs[j])
				maxs[j] = vertices[j+d*(i-1)]
			end
			if(vertices[j+d*(i-1)] < mins[j])
				mins[j] = vertices[j+d*(i-1)]
			end
		end
	end
	
	return (mins,maxs)
end

#Questa funzione computa un dizionario avente come chiave una coordinata [cmin,cmax], e
#come valore un array contenente gli indici di tutte le celle incidenti sulla chiave. 
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	#Per ogni bounding box...
	l = length(bboxes)
	for h=1:l
		#La chiave del dizionario è [cmin,cmax]
		key = bboxes[h][coord,:]
		#Se il dizionario non ha la chiave, creo la entry..
		if !haskey(boxdict,key)
			boxdict[key] = [h]
		else #Altrimenti pusho la cella, in quanto condividerà [cmin,cmax] con altre celle
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

#Questa funzione computa le intersezioni di celle facendo una ricerca efficiente sugli 
#intervalTrees. index=1 per assex, index=2 per assey, index=3 per assez
function boxcovering(bboxes, index, tree)
	#Inizializzo array vuoti per tutti i box
	n = length(bboxes)
	covers = Array{Array{Int64,1}}(undef, n)
	@inbounds @threads for k=1:n
		covers[k] = []
	end
	#Per ogni bbox....
	@inbounds for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		#Faccio una query all'interval tree su un intervallo 
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		#Tutte le celle che trovo le appendo all'array del box
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

function skel_merge(V1::Lar.Points, EV1::Lar.ChainOp, V2::Lar.Points, EV2::Lar.ChainOp)
    V = [V1; V2]
    EV = blockdiag(EV1,EV2)
    return V, EV
end

skel_merge (generic function with 2 methods)

## Dati in input

In [29]:
b=[[],[]]
EV=[[1,1]]

for i=1:60
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(1.0 + i*2.0))
           push!(b[1],(1.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           push!(b[1],(4.0 + i*2.0))
           push!(b[2],(4.0 + i*2.0))
           
           push!(EV,[1+4*(i-1),2+4*(i-1)])
           push!(EV,[1+4*(i-1),3+4*(i-1)])
           push!(EV,[2+4*(i-1),4+4*(i-1)])
           push!(EV,[3+4*(i-1),4+4*(i-1)])
end

V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
filter!(e->e!=[1,1],EV)

W = convert(Lar.Points, V')
cop_EV = Lar.coboundary_0(EV::Lar.Cells)
cop_EW = convert(Lar.ChainOp, cop_EV)



240×240 SparseMatrixCSC{Int8,Int64} with 480 stored entries:
  [1  ,   1]  =  -1
  [2  ,   1]  =  -1
  [1  ,   2]  =  1
  [3  ,   2]  =  -1
  [2  ,   3]  =  1
  [4  ,   3]  =  -1
  [3  ,   4]  =  1
  [4  ,   4]  =  1
  [5  ,   5]  =  -1
  [6  ,   5]  =  -1
  [5  ,   6]  =  1
  [7  ,   6]  =  -1
  ⋮
  [235, 234]  =  -1
  [234, 235]  =  1
  [236, 235]  =  -1
  [235, 236]  =  1
  [236, 236]  =  1
  [237, 237]  =  -1
  [238, 237]  =  -1
  [237, 238]  =  1
  [239, 238]  =  -1
  [238, 239]  =  1
  [240, 239]  =  -1
  [239, 240]  =  1
  [240, 240]  =  1

## Benchmark vecchia funzione

In [30]:
@btime planar_arrangement_1(W,cop_EW)

  67.883 ms (1002504 allocations: 37.49 MiB)


([3.0 3.0; 6.0 3.0; … ; 121.0 124.0; 124.0 124.0], 
  [1  ,   1]  =  1
  [2  ,   1]  =  1
  [1  ,   2]  =  1
  [3  ,   2]  =  1
  [2  ,   3]  =  1
  [5  ,   3]  =  1
  [4  ,   4]  =  1
  [6  ,   4]  =  1
  [3  ,   5]  =  1
  [4  ,   5]  =  1
  [7  ,   5]  =  1
  [8  ,   5]  =  1
  ⋮
  [472, 353]  =  1
  [469, 354]  =  1
  [470, 354]  =  1
  [473, 354]  =  1
  [474, 354]  =  1
  [471, 355]  =  1
  [473, 355]  =  1
  [472, 356]  =  1
  [475, 356]  =  1
  [474, 357]  =  1
  [476, 357]  =  1
  [475, 358]  =  1
  [476, 358]  =  1, 0-element SparseVector{Int8,Int64} with 0 stored entries, [[1], [2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18]  …  [459, 460], [461, 462], [463, 464], [465, 466], [467, 468], [469, 470], [471, 472], [473, 474], [475], [476]])

## Controllo se la funzione è type unstable 

In [31]:
@code_warntype(planar_arrangement_1(W,cop_EW))

Variables
  #self#::Core.Compiler.Const(planar_arrangement_1, false)
  V::Array{Float64,2}
  copEV::SparseMatrixCSC{Int8,Int64}

Body::Tuple{Any,SparseMatrixCSC{Int8,Int64},SparseVector{Int8,Int64},Array{Array{Int64,1},1}}
1 ─ %1 = Main.spzeros(Main.Int8, 0)::Core.Compiler.PartialStruct(SparseVector{Int8,Int64}, Any[Core.Compiler.Const(0, false), Array{Int64,1}, Array{Int8,1}])
│   %2 = (#self#)(V, copEV, %1, false, false)::Tuple{Any,SparseMatrixCSC{Int8,Int64},SparseVector{Int8,Int64},Array{Array{Int64,1},1}}
└──      return %2


La funzione NON è type unstable poichè riporta l'output:

    Body::Tuple{Any,SparseMatrixCSC{Int8,Int64},SparseVector{Int8,Int64},Array{Array{Int64,1},1}}

## Parallelizzazione cicli con i threads

Non abbiamo ritenuto idoneo parallelizzare questo codice (la funzione di Lar già prevede la parte con il multiproc = true), in quanto sia il frag_edge che l'etichettatura dei vertici e lo skel-merge non sono tasks parallelizzabili. Usare i thread senza stravolgere la struttura del codice comporta risultati errati.

In [37]:
using Base.Threads
function planar_arrangement_1( V,copEV,sigma::Lar.Chain=spzeros(Int8,0),
                               return_edge_map::Bool=false,multiproc::Bool=false)
	# data structures initialization
	edgenum = size(copEV, 1)
	edge_map = Array{Array{Int, 1}, 1}(undef,edgenum)
	rV = Lar.Points(zeros(0, 2))
	rEV = SparseArrays.spzeros(Int8, 0, 0)
	finalcells_num = 0

	# spaceindex computation
	model = (convert(Lar.Points,V'),Lar.cop2lar(copEV))
	bigPI = spaceindex(model::Lar.LAR)

        c = Threads.Condition()
        # sequential (iterative) processing of edge fragmentation
        @threads for i in 1:edgenum
            v, ev = frag_edge(V, copEV, i, bigPI)
            lock(c)
            newedges_nums = map(x->x+finalcells_num, collect(1:size(ev, 1)))
            edge_map[i] = newedges_nums
            finalcells_num += size(ev, 1)
            rV = convert(Lar.Points, rV)
            rV, rEV = skel_merge(rV, rEV, v, ev)
            unlock(c)
        end
    
    # merging of close vertices and edges (2D congruence)
    V, copEV = rV, rEV
    V, copEV = merge_vertices!(V, copEV, edge_map)
	return V,copEV,sigma,edge_map
end

nt = nthreads()
println("Numero threads allocati : $nt")
@btime planar_arrangement_1(W,cop_EW)

Numero threads allocati : 2
  59.852 ms (986188 allocations: 35.89 MiB)


([3.0 3.0; 6.0 3.0; … ; 64.0 64.0; 124.0 124.0], 
  [1  ,   1]  =  1
  [4  ,   1]  =  1
  [1  ,   2]  =  1
  [7  ,   2]  =  1
  [2  ,   3]  =  1
  [5  ,   3]  =  1
  [3  ,   4]  =  1
  [9  ,   4]  =  1
  [2  ,   5]  =  1
  [3  ,   5]  =  1
  [471,   5]  =  1
  [472,   5]  =  1
  ⋮
  [471, 352]  =  1
  [465, 353]  =  1
  [469, 353]  =  1
  [466, 354]  =  1
  [473, 354]  =  1
  [468, 355]  =  1
  [474, 355]  =  1
  [470, 356]  =  1
  [476, 356]  =  1
  [472, 357]  =  1
  [475, 357]  =  1
  [473, 358]  =  1
  [476, 358]  =  1, 0-element SparseVector{Int8,Int64} with 0 stored entries, [[1], [4], [7, 8], [11, 12], [15, 16], [19, 20], [23, 24], [27, 28], [31, 32], [35, 36]  …  [441, 442], [445, 446], [449, 450], [453, 454], [457, 458], [461, 462], [465, 466], [469, 470], [473], [476]])

Si nota che con l'uso dei threads il risultato dell'edge map è diverso da quello iniziale, pertanto errato.